ticket_master.ipynb

In [5]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
# timdelta
from datetime import datetime, timedelta

load_dotenv()

True

In [7]:
def get_sports_events_df(city="San Diego"):    
    # get api key from .env
    api_key = os.getenv("ticketmaster_api_key")
    # set date range
    start_date = datetime.now()
    end_date = start_date + timedelta(days=180)  # 6 months
    # format dates for api
    start_str = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_str = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    # endpoint
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    # params - specifically for sports events
    params = {
        "apikey": api_key,
        "size": 500,  # increased size to get more events
        "city": city, 
        "startDateTime": start_str,
        "endDateTime": end_str,
        "segmentName": "Sports",  # Filter for sports events
        "sort": "date,asc",
        "includeTBA": "no",  # Exclude TBA events
        "includeTBD": "no"   # Exclude TBD events
    }
    try:
        # make api request
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        # check for events present
        if '_embedded' not in data or 'events' not in data['_embedded']:
            return pd.DataFrame()  # Return empty DataFrame if no events

        # extract events
        events = data['_embedded']['events']
        # empty list to store event data
        event_data = []
        # loop through events
        for event in events:
            # filter for events that are on sale
            status = event.get('dates', {}).get('status', {}).get('code')
            if status != 'onsale':
                continue  

            # extract venue info safely
            venue_info = event.get('_embedded', {}).get('venues', [{}])[0]
            # get more specific sports info
            classifications = event.get('classifications', [{}])[0]
            sport_type = classifications.get('subGenre', {}).get('name') or classifications.get('genre', {}).get('name')
            # dictionary for each event
            event_dict = {
                'name': event.get('name'),
                'date': event.get('dates', {}).get('start', {}).get('localDate'),
                'time': event.get('dates', {}).get('start', {}).get('localTime'),
                'venue': venue_info.get('name'),
                'sport_type': sport_type,
                'min_price': None,
                'max_price': None,
                'tickets_url': event.get('url')
            }
            
            # add price min max if available
            if 'priceRanges' in event:
                event_dict['min_price'] = event['priceRanges'][0].get('min')
                event_dict['max_price'] = event['priceRanges'][0].get('max')
                
            event_data.append(event_dict)

            # filter out 'Tours'
            event_data = [event for event in event_data if 'Tours' not in event['name']]
        
        # create df from event data
        df = pd.DataFrame(event_data)
        
        # clean up date column
        if not df.empty and 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        
        # sort by date
        if not df.empty:
            df = df.sort_values('date')

        return df

    # error handling    
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err.response.status_code}")
        return pd.DataFrame()
    except Exception as err:
        print(f"Error occurred: {err}")
        return pd.DataFrame()

if __name__ == "__main__":
    # GET sports events DataFrame
    sports_events_df = get_sports_events_df("San Diego")
    
    if not sports_events_df.empty:
        print(f"\nFound {len(sports_events_df)} upcoming sports events in San Diego:")
        display(sports_events_df)
    else:
        print("No events or error occurred.")

HTTP error occurred: 400
No events or error occurred.
